source : https://github.com/mila-udem/blocks-examples/blob/master/reverse_words/__init__.py

In [9]:
import theano
import numpy

from theano import tensor
from fuel.streams import ServerDataStream

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from models.ann_2_blocks import build_model
from blocks.algorithms import GradientDescent, Adam
from blocks.graph import ComputationGraph
from blocks.model import Model
import csv

LOAD COMPUTATIONAL MODEL

In [10]:
images           = tensor.dtensor4('images')
images_test      = tensor.dtensor4('images_test')
labels           = tensor.lmatrix('labels')
cost             = build_model(images.dimshuffle(0,3,1,2), labels)

In [11]:
from blocks.model import Model
from blocks.serialization import load_parameters, load
import pickle

In [12]:
#main_loop = load(open("../train", "rb"))
#sample = theano.function([images_test], main_loop.model.get_top_bricks()[0].apply(images_test.dimshuffle(0,3,1,2)))
dict = {0: 'safe driving', 1: 'texting-right', 2: 'talking on the phone - right', 3: 'texting - left',
           4: 'talking on the phone', 5: 'operating the radio', 6: 'drinking', 7: 'reaching behind',
           8: 'hair and makeup', 9: 'talking to passenger'}

In [13]:
parameters = load_parameters(open("../train", "rb"))
model = Model(cost)
model.set_parameter_values(parameters)
sample = theano.function([images_test], model.get_top_bricks()[0].apply(images_test.dimshuffle(0,3,1,2)))

Upload testing data

In [14]:
from fuel.datasets.hdf5 import H5PYDataset
from fuel.schemes import SequentialScheme
from fuel.streams import DataStream
from functions.custom_transformers import Normalize, Cast
submit_set = H5PYDataset('../data/data.hdf5', which_sets=('submit',), subset=slice(0, 79726),)

In [20]:
stream = DataStream.default_stream(
    submit_set,
    iteration_scheme=SequentialScheme(submit_set.num_examples, 500)
)
stream_normalized = Normalize(stream)
stream_casted     = Cast(stream_normalized, 'floatX')

Create submission file

In [21]:
stream.reset()
f = open('submit_1.csv', 'ab'); spamwriter = csv.writer(f, delimiter=',')
spamwriter.writerow(['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
count      = 0
it = stream_casted.get_epoch_iterator()
for data in it:
    labels = sample(data[0])
    for i in range(labels.shape[0]):
        spamwriter.writerow(['img_' + str(data[1][i][0]) + '.jpg'] + list(labels[i]))
        count += 1
assert(count == 79726)

In [19]:
count

79726